In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import Dataset, DataLoader, default_convert
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datetime import datetime

In [6]:
class GazeEstimationDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.eyes_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.eyes_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.eyes_frame.iloc[idx, 0]
        image = io.imread(img_name)
        image = image/255
        rgb_img = np.repeat(image[..., np.newaxis], 3, -1)
        #print(rgb_img.shape, "image shape")
        face_img_coor = np.fromstring(self.eyes_frame.iloc[idx, 3][1:int(len(self.eyes_frame.iloc[idx, 3]) - 1)],
                                      sep=',',
                                      dtype=int)
        roi_eyes_coor = np.fromstring(self.eyes_frame.iloc[idx, 4][1:int(len(self.eyes_frame.iloc[idx, 4]) - 1)],
                                      sep=',', dtype=int)
        face_img = rgb_img[face_img_coor[0]: face_img_coor[0] + face_img_coor[2],
                   face_img_coor[1]: face_img_coor[1] + face_img_coor[3], :]

        opt_flow_name = self.eyes_frame.iloc[idx, 5]
        opt_flow = pd.read_csv(opt_flow_name)
        sample = {'name': img_name, 'face': face_img, 'face_coor': face_img_coor, 'eyes_coor': roi_eyes_coor,
                  'x': self.eyes_frame.iloc[idx, 1], 'y': self.eyes_frame.iloc[idx, 2]}
        '''opt_flow': opt_flow'''
        if self.transform:
            sample = self.transform(sample)
        return sample


class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, eyes = sample['face'], sample['eyes_coor']

        h, w = image.shape[:2]
        # if isinstance(self.output_size, int):
        #    if h > w:
        #        new_h, new_w = self.output_size * h / w, self.output_size
        #    else:
        #        new_h, new_w = self.output_size, self.output_size * w / h
        # else:
        new_h, new_w = self.output_size, self.output_size
        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        for i in range(2):
            eyes[i] = eyes[i] * new_w / w
        for i in range(2, 4, 1):
            eyes[i] = eyes[i] * new_h / h
        eyes[4] = eyes[4] * new_w / w
        eyes[5] = eyes[5] * new_h / h

        x, y = sample['x'], sample['y']
        # opt_flow = sample['opt_flow']

        return {'face': img, 'eyes_coor': eyes, 'x': x, 'y': y}

    '''opt_flow': opt_flow'''

class ToTensor(object):
    def __call__(self, sample):
        image, eyes = sample['face'], sample['eyes_coor']
        x, y = (sample['x'] + 800) / 1600, (sample['y'] + 800) / 1600
        # opt_flow = sample['opt_flow']
        image = image.transpose((2, 0, 1))
        return {'face': torch.from_numpy(image).type(torch.DoubleTensor),
                'eyes_coor': torch.from_numpy(eyes).type(torch.DoubleTensor),
                'gt_coor': torch.tensor([x, y]).type(torch.DoubleTensor)}

    '''opt_flow': torch.from_numpy(opt_flow.values)'''


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(59536, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
dataset = GazeEstimationDataset(csv_file="full_face/total.csv", root_dir="")
transformed_dataset = GazeEstimationDataset(csv_file="full_face/total.csv", root_dir="",
                                            transform=transforms.Compose([Rescale(256), ToTensor()]))
start = datetime.now()
print("start splitting at: ", start)
train, test = train_test_split(transformed_dataset)
end = datetime.now()
print("splitting done after: ", end - start)

start splitting at:  2022-06-29 21:08:58.430322


In [ ]:
batch_size = 20
trainloader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=0)

net = Net()
net = net.double()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
n = 7
for epoch in range(n):
    with tqdm(trainloader, unit="batch") as tepoch:
        running_loss = 0.0
        for data in tepoch:
            # print(data['face'].shape)
            tepoch.set_description(f"Epoch {epoch}")
            inputs = data['face']
            # data['eyes_coor']
            # data['opt_flow']
            labels = data['gt_coor']

            optimizer.zero_grad()

            output = net(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            correct = (abs(output - labels)).sum().item()
            accuracy = correct / batch_size

            tepoch.set_postfix(loss=loss.item(), accuracy=100-accuracy)
            time.sleep(0.1)
print('done')
path = '../trained_ts.pth'
torch.save(net.state_dict(), path)

In [ ]:
total = 0
correct = 0

with torch.no_grad():
    for data in test:
        image, labels = data
        output = net(image)
        _, predicted = torch.max(output.data, 1)
        total += 1
        correct += abs(predicted-labels)
print('overall ', correct/total)